Project Overview and Goals

# 🎬 Movie Recommender System (MVP)

## 🧠 Project Overview
This is a beginner-friendly **Movie Recommender System** built using content-based filtering. It aims to suggest movies similar to a user's input based on features like genres, keywords, and plot metadata.

The project follows the **Software Development Life Cycle (SDLC)** to demonstrate clean design, modularity, and progressive improvement.

---

## 📌 Goals

### ✅ Minimum Viable Product (MVP)
- Load and explore a movie metadata dataset (e.g., from TMDb or IMDB)
- Preprocess features like genres, overview, cast, and keywords
- Create a content-based recommender using TF-IDF and cosine similarity
- Allow a user to input a movie and receive top 5 similar recommendations

### 🔄 Expansion Ideas
- Add collaborative filtering using user ratings
- Use BERT or sentence transformers for better plot similarity
- Personalize recommendations using user history or favorites
- Build a Streamlit or Flask app interface

---

## 📁 Tech Stack
- **Python**
- **Pandas**, **NumPy** for data handling
- **Scikit-learn** for vectorization and similarity
- **Streamlit** (optional) for frontend
- **TMDb API** or Kaggle dataset as data source

---

> This notebook is designed to be self-contained and modular for easy iteration and expansion.


Data Loading

Preprocessing 

TF-IDF Vectorization

Similarity Calculation

Recommender Function

UI

Testing

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")


In [2]:
import requests

def search_movie(query, api_key):
    url = f"https://api.themoviedb.org/3/search/movie"
    params = {
        "api_key": api_key,
        "query": query
    }
    response = requests.get(url, params=params)
    return response.json()


In [7]:
result = search_movie("Inception", API_KEY)

In [8]:
movie_id = result['results'][0]['id']

In [9]:
def get_tmdb_recommendations(movie_id, api_key):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/recommendations"
    params = {
        "api_key": api_key
    }
    response = requests.get(url, params=params)
    return response.json()


In [10]:
def get_movie_details(movie_id, api_key):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    params = {
        "api_key": api_key
    }
    response = requests.get(url, params=params)
    return response.json()
